In [1]:
from selenium import webdriver
from time import sleep
import config_essentials
from bs4 import BeautifulSoup
import pandas as pd
import os

driver=webdriver.Chrome(executable_path=config_essentials.CHROMELOC)


C:\Users\elenc\AppData\Local\Temp/ipykernel_11992/4250501570.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path=config_essentials.CHROMELOC)


In [ ]:

# head over to garage and moving sales
driver.get("https://gab.com/auth/sign_in")
#driver.get("https://accounts.craigslist.org/login")
#sleep(3)

login_form = driver.find_element_by_id("user_email").send_keys(cdconfig.UNAME)
login_form = driver.find_element_by_id("user_password").send_keys(cdconfig.PWORD)
try_click = driver.find_element_by_id("picview").click()
button


In [ ]:

sleep(1)
try_click = driver.find_element_by_id("picview").click()
sleep(2)
#login_form = driver.find_element_by_name("inputEmailHandle").send_keys(cdconfig.UNAME)

# gather up distinct listing urls to crawl
listing_urls = []
bs = BeautifulSoup(driver.page_source, 'html.parser')
block = bs.find(id="search-results")
for listing in block.find_all('a'):
    lurl= listing['href']
    if (lurl != '#') & (lurl not in listing_urls):
        listing_urls.append(lurl)
    
#login_form = driver.find_element_by_name("inputPassword").send_keys(cdconfig.PWORD)

#psource = driver.page_source
#now we could throw it into BeautifulSoup!
#print(psource)